# 🎥 YouTube'dan Hugging Face'e Toplu Veri Seti Yükleme

Bu notebook, YouTube videolarını toplu olarak indirip işleyerek Hugging Face'e yükler.

📝 Kullanım:
1. Runtime > Run all seçeneğini seçin
2. Hugging Face token'ınızı girin
3. YouTube URL'lerini listeleyin
4. İşlemin tamamlanmasını bekleyin

In [ ]:
# @title 🔧 Kurulum ve Bağımlılıklar

!pip install yt-dlp pydub webvtt-py datasets transformers librosa huggingface_hub python-dotenv

import os
import subprocess
import time
from getpass import getpass
from typing import List

# Klasörleri oluştur
!mkdir -p /content/output/audio
!mkdir -p /content/output/json
!mkdir -p /content/output/spectrogram

# Gerekli scriptleri indir
!wget https://raw.githubusercontent.com/zinderud/sayha/main/youtube_splitter_tr.py
!wget https://raw.githubusercontent.com/zinderud/sayha/main/processed_dataset.py
!wget https://raw.githubusercontent.com/zinderud/sayha/main/upload_to_huggingface.py

In [ ]:
# @title 🛠 İşlev Tanımlamaları

def process_youtube_video(youtube_url: str) -> bool:
    try:
        # YouTube videosunu işle
        !python youtube_splitter_tr.py {youtube_url}
        
        # Veri setini işle
        !python processed_dataset.py
        
        # Hugging Face'e yükle
        !python upload_to_huggingface.py
        
        print(f"✅ {youtube_url} başarıyla işlendi!")
        return True
        
    except Exception as e:
        print(f"❌ Hata oluştu ({youtube_url}): {e}")
        return False
    finally:
        # Geçici dosyaları temizle
        !rm -rf /content/output/audio/*
        !rm -rf /content/output/json/*
        !rm -rf /content/output/spectrogram/*

def process_video_list(youtube_urls: List[str]):
    total_videos = len(youtube_urls)
    successful_videos = 0
    failed_videos = 0

    print(f"\nToplam {total_videos} video işlenecek...")
    print("-" * 50)

    for index, url in enumerate(youtube_urls, 1):
        print(f"\n📽 Video {index}/{total_videos} işleniyor...")
        print(f"🔗 URL: {url}")
        
        if process_youtube_video(url):
            successful_videos += 1
        else:
            failed_videos += 1
        
        # Son video değilse bekle
        if index < total_videos:
            print("\n⏳ Bir sonraki video için 60 saniye bekleniyor...")
            time.sleep(60)
    
    print("\n" + "=" * 50)
    print("İşlem Özeti:")
    print(f"✅ Başarılı: {successful_videos}")
    print(f"❌ Başarısız: {failed_videos}")
    print(f"📊 Başarı Oranı: {(successful_videos/total_videos)*100:.1f}%")
    print("=" * 50)

In [ ]:
# @title 🔑 Hugging Face Token'ı Ayarla

token = getpass('Hugging Face Token: ')
os.environ['HUGGINGFACE_TOKEN'] = token

In [ ]:
# @title 🎬 Video İşleme

# Video URL'lerini buraya ekleyin
youtube_urls = [
    # Örnek:
    # "https://www.youtube.com/watch?v=video1",
    # "https://www.youtube.com/watch?v=video2",
]

# URL'leri işle
if youtube_urls:
    process_video_list(youtube_urls)
else:
    print("❌ En az bir YouTube URL'si eklemelisiniz!")